# Inundation History Tool v2

* **Compatability:** Notebook currently compatible with the `DEA Sandbox` environment
* **Products used:** 
['ga_ls5t_ard_3'](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
['ga_ls7e_ard_3'](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
['ga_ls8c_ard_3'](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3),
['s2a_ard_granule'](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
['s2b_ard_granule'](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3),
[BoM Water Data Online](http://www.bom.gov.au/waterdata/)


## Purpose


This tool is used for associating river flow rates with Landsat and Sentinel satellite passes. It filters satellite passes within defined flow bands of interest, removes poor quality satellite data, and also applies a filter to those images on the rising/falling limb. The images are then analysed using the Fisher index, MNDWI, and a False colour (Burnsie Index) for water identification in the landscape. 

The tool outputs the images as NetCDF files for use in GIS software.

## Quick use notes

For **issues relating to the script, a tutorial, or feedback** please contact Martin Job at martin.job@mdba.gov.au or David Weldrake at david.weldrake@mdba.gov.au
1. Press shift + enter on cells until a map appears, select the gauge of interest from the map. Make sure to click "done" after you have selected your gauge.
2. Use the dashboard under the map to input your selections. Start by defining the flow band of interest. You then have three options for defining the extents of your location of interest. Extents can be defined by uploading a shapefile, typing in the coordinates manually, or using the coordinates of the gauge you selected in the previous step. 
3. Press shift + enter until you get to 'PART 2'. Here you can toggle the satellite passes you wish to analyse and export based on their position on the hydrograph.
5. Download the NetCDF files for import into your preferred GIS software, and inspect the thumbail images for each satellite flyover.
6. OPTIONAL: download any of the plots of interest.
7. OPTIONAL: Download the csv, containing the date of the flyover, gauge reading, and whether you considered it a pass or fail.
8. Go back to the gauge selecting map at the top of the page, re-run this cell, and select a new gauge, repeat the process to get this new data

### Load packages
Load key Python packages and supporting functions for the analysis. This notebook relies on modules called `dea_bom` and `dea_datahandling`, which are located in the `Scripts` directory, and the `IHT` module located in the `iht_modules` directory. 

### Version 2 changes:

- Integrate Sentinel 2
- Additional band combinations (the false colour 'Burnsie index')
- Implementing widgets for user inputs
- Automatic uploading to the MDBA arcgis portal

In [1]:
import sys
import os
import pickle
import xarray as xr
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime

import datacube
from datacube import Datacube
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.drivers.netcdf import write_dataset_to_netcdf

%matplotlib inline

import warnings
warnings.filterwarnings('ignore', module='datacube')
%load_ext autoreload
%autoreload 2

sys.path.append(os.path.abspath('/home/jovyan/Scripts'))
import dea_bom
from dea_datahandling import load_ard
from iht_modules import iht

import plotly.graph_objects as go
import matplotlib.dates
import plotly.offline as py

import geopandas as gpd

import ipywidgets as widgets

### Preparing and loading the gauge information from BOM's Water Data Online 

In [3]:
stations_pkl = Path('/home/jovyan/Supplementary_data/Inundation_mapping/stations.pkl')

# If cache exists, get station data from cache
if stations_pkl.exists():
    print('Loading from cache')
    stations = pickle.load(open(str(stations_pkl), 'rb'))
else:
    print('Fetching from BoM')
    stations = dea_bom.get_stations()
    pickle.dump(stations, open(str(stations_pkl), 'wb'))

# Filter list to stations with available data
stations_with_data = pickle.load(open(str('/home/jovyan/Supplementary_data/Inundation_mapping/stations_with_data.pkl'), 'rb'))
stations = [i for i in stations if i.name in stations_with_data]

# Preview the first five stations loaded
print(f'{len(stations)} stations loaded; e.g.:')
stations[:5]

Loading from cache
4303 stations loaded; e.g.:


[namespace(name='15 MILE @ GRETA STH',
           url='http://bom.gov.au/waterdata/services/stations/403213',
           pos=(-36.61945775, 146.24407214)),
 namespace(name='15 MILE @ WANGARATTA',
           url='http://bom.gov.au/waterdata/services/stations/403239',
           pos=(-36.36666667, 146.2833333)),
 namespace(name='16 Mile Waterhole',
           url='http://bom.gov.au/waterdata/services/stations/913010A',
           pos=(-18.876921, 139.360487)),
 namespace(name='163 Clifton Rd',
           url='http://bom.gov.au/waterdata/services/stations/6131318',
           pos=(-32.97808, 115.90111)),
 namespace(name='18 Mile Swamp HorseX',
           url='http://bom.gov.au/waterdata/services/stations/144005A',
           pos=(-27.49561971, 153.50836409))]

In [ ]:
gauge_data, station = dea_bom.ui_select_station(stations,
                                                zoom=5,
                                                center=(-34.72, 143.17));

### Input user preferences for lat, lon, satellite buffer, and flow thresholds

In [ ]:
iht.iht_dashboard

In [ ]:
yaxis_lower_parameter, yaxis_higher_parameter = iht.get_flow_bounds(iht.min_flow.value, 
                                                                    iht.max_flow.value)

lat_low, lat_high, lon_low, lon_high = iht.get_coords(iht.check_gauge.value,
                                                  iht.check_own.value,
                                                  iht.input_lat.value,
                                                  iht.input_lon.value,
                                                  iht.check_shapefile.value, 
                                                  iht.shapefile_loc.value, 
                                                  iht.buffer.value,
                                                  station.pos)

### Loading the Landsat and Sentinel passes for the location of interest

The second cell in this block will take up to 5 minutes to run, as it is loading the satellite passes, applying a cloud filter and filtering out passes with poor coverage

In [ ]:
landsat_dc = datacube.Datacube(app='Loading Landsat IHT')

todays_date = str(datetime.today().strftime('%Y-%m-%d')) 

landsat_query = {
    'x': (lon_low, lon_high),
    'y': (lat_low, lat_high),
    'time': ('1988-01-01', todays_date),
    'measurements': ["nbart_red"],
    'output_crs': 'EPSG:3577',
    'resolution': (-30, 30),
    'group_by': 'solar_day'
}

# Load available data from all three Landsat satellites using the above query
# Please note the two available options below. You can uncomment the min_gooddata and 
# change this value to reflect the minimum good quality pixels to include (0.6 = 60%)
# Changing the ls7_slc_off variable to True will include ls7 passes with the slc failure
landsat_ds = load_ard(dc=landsat_dc, 
              products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'],
              mask_pixel_quality=True,
              min_gooddata=0.60, #Toggle number between 0 and 1 (default is 0.6)
              ls7_slc_off=False, # Change to True to include Ls7 passes with slc failure
              dask_chunks={}, 
              **landsat_query)

In [ ]:
sentinel_dc = datacube.Datacube(app='Loading sentinel IHT')

sentinel_query = {
    'x': (lon_low, lon_high),
    'y': (lat_low, lat_high),
    'time': ('2015-06-23', todays_date),
    'measurements': ['nbart_red'],
    'output_crs': 'EPSG:3577',
    'resolution': (-10, 10),
    'group_by': 'solar_day'
}

# Load available data from both Sentinel 2 satellites
sentinel_ds = load_ard(dc=sentinel_dc,
              products=['s2a_ard_granule', 's2b_ard_granule'],
              min_gooddata=0.10,
              mask_pixel_quality=True,         
              dask_chunks={},
              **sentinel_query)

### Filtering out passes for dates within the flow bands of interest and
 ### Merging the satellite passes with the gauge data

In [ ]:
def gauge_data_cleaner(input_data):
    '''Ingests a pd.df of flow timeseries, 
    converts data types and measurements,
    returns clean gauge data'''
    gauge_df = input_data.copy(deep = True)
    gauge_df["Value"] = pd.to_numeric(gauge_df["Value"], downcast="float")
    gauge_df['Value'] = gauge_df['Value']*86.4
    gauge_df.index = gauge_df.index.normalize()
    
    return gauge_df

clean_gauge_data = gauge_data_cleaner(gauge_data)

In [ ]:
def merge_satellite_with_gauge(sat_input, gauge_input):
    '''Ingests gauge data and satellite data xr,
    identifies where on the gauge data there are passes and works out how many fall within the
    flow bands of interest,
    returns the dates, how many there are, and the xarray of the merged data
    '''
    gauge_data_xr = gauge_input.to_xarray() 
    merged_data = gauge_data_xr.interp(Timestamp=sat_input.time)
    specified_satellite_passes = merged_data.where(((merged_data.Value > yaxis_lower_parameter) &\
                                                    (merged_data.Value < yaxis_higher_parameter)), 
                                                   drop=True)
    specified_satellite_passes = specified_satellite_passes.drop('Timestamp')
    date_list = specified_satellite_passes.time.values
    
    how_many = specified_satellite_passes.time.shape[0]
    
    return date_list, how_many, merged_data

ls_date_list, ls_count, ls_merged_data = merge_satellite_with_gauge(landsat_ds, clean_gauge_data)  
s_date_list, s_count, s_merged_data = merge_satellite_with_gauge(sentinel_ds, clean_gauge_data)

#Check how many passes you are about to load. 
#Loading over 400 passes may cause the kernal to crash
print("You are about to load {} landsat passes".format(ls_count))
print("You are about to load {} sentinel passes".format(s_count))

In [ ]:
def compute_passes(input_sat_passes, input_date_list):
    '''Ingests satellite data dask array and the required dates,
    computes the dask array for these dates only'''
    
    computed = input_sat_passes.sel(time=input_date_list).compute()
    
    return computed
    
ls_specified_passes = compute_passes(landsat_ds, ls_date_list)
s_specified_passes = compute_passes(sentinel_ds, s_date_list)

In [ ]:
def convert_to_pandas(input_merged_data, input_specified_passes):
    ''' Converts to dataframe 
    returns the dataframe with the gauge reading at the time of the satellite pass '''
    merged_data_pd = input_merged_data.to_dataframe()
    
    all_specified_passes_pd = input_specified_passes.time.to_dataframe()
    all_specified_passes_pd = all_specified_passes_pd.rename(columns = {'time': 'date'})
    
    all_merged_data = pd.merge(all_specified_passes_pd, merged_data_pd, left_on= 'time', 
                                right_index=True, how='inner')
    all_merged_data = all_merged_data.drop(columns='date')
    all_merged_data = all_merged_data.drop(columns='Timestamp')
    all_merged_data.index = all_merged_data.index.normalize()
    all_merged_data.index.name = 'date'
    
    return all_merged_data
    
ls_all_merged_data = convert_to_pandas(ls_merged_data, ls_specified_passes)
s_all_merged_data = convert_to_pandas(s_merged_data, s_specified_passes)

In [ ]:
def add_flow_bands_to_gauge_data(input_gauge_data):
    ''' Adds the flow bands of interest to the gauge dataframe for graphing purposes '''

    graph_gauge_df = input_gauge_data.copy(deep=True)
    
    graph_gauge_df['y_lower'] = yaxis_lower_parameter
    graph_gauge_df['y_higher'] = yaxis_higher_parameter
    
    return graph_gauge_df

In [ ]:
def graph_all(input_gauge_data, input_ls, input_s):
    ''' Function to generate a graph showing the satellite passes relative to where they occur
    on the hydrograph '''
    
    # Set up the dataframe:
    graph_gauge_df = add_flow_bands_to_gauge_data(input_gauge_data)
    
    fig_all = go.Figure()
    
    # Add gauge data:
    fig_all.add_trace(go.Scatter(name = 'Gauge data',
                                x = graph_gauge_df.index,
                                y = graph_gauge_df['Value'],
                                mode = 'lines',
                                line = dict(color = 'royalblue', width = 1),
                                hovertemplate = 'flow: %{y:.2f}<extra></extra>' +
                                 '<br><b>Date</b>: %{x}<br>'
                                )
                     )
    
    # Add landsat:
    fig_all.add_trace(go.Scatter(name = 'Landsat pass',
                                x = input_ls.index,
                                y = input_ls['Value'],
                                mode = 'markers',
                                line = dict(color = 'green', width = 2), 
                                hovertemplate = 'Landsat pass <extra></extra>' +
                                '<br><b>Date</b>: %{x}<br>',
                                marker_size = 9
                                )
                     )
    
    # Add sentinel:
    fig_all.add_trace(go.Scatter(name = 'Sentinel pass',
                                x = input_s.index,
                                y = input_s['Value'],
                                mode = 'markers',
                                line = dict(color = 'lightseagreen', width = 2), 
                                hovertemplate = 'Sentinel pass <extra></extra>' +
                                 '<br><b>Date</b>: %{x}<br>',
                                marker_symbol = 'cross',
                                marker_size = 9
                                )
                     )
    
    # Adding the lower flow bounds:
    fig_all.add_trace(go.Scatter(name = 'lower flow bound',
                                x = graph_gauge_df.index,
                                y = graph_gauge_df['y_lower'],
                                mode = 'lines',
                                line = dict(color = 'black', width = 2)
                                )
                     )
    
    # Adding the upper flow bounds:
    fig_all.add_trace(go.Scatter(name = 'upper flow bound',
                                x = graph_gauge_df.index,
                                y = graph_gauge_df['y_higher'],
                                mode = 'lines',
                                line = dict(color = 'black', width = 2)
                                )
                     )

    fig_all.update_layout(hovermode="closest",
                                     title= 'Landsat and Sentinel passes within the flow band of interest',
                                     xaxis_title='Date',
                                     yaxis_title='Flow at gauge (ML/Day)',
                                     font=dict(family='Calibri', size=16, color='black'),
                                     paper_bgcolor='white',
                                     plot_bgcolor='white',
                                     hoverlabel=dict(
                                         bgcolor="white",
                                         font_size=16,
                                         font_family="Rockwell")) 
    
    return fig_all

fig_all = graph_all(clean_gauge_data, ls_all_merged_data, s_all_merged_data)
fig_all

### Now separate the gauge data into 2 lists: rising and falling. Check the graph in the output to see how well the data was separated

This box will check whether the gauge-reading 21 (can change with user input) days after the satellite pass was higher or lower than the day of the satellite pass. The multiplier represents by how much more the water should be lower or higher to be considered a significant change. This has been defined as 1 for simplicity (can also change with user input). It puts the pass either into the rising or falling list accordingly. It runs a loop to do this for every single pass. The output will tell you how many passes you got in each list and show you how the passes were catagorised on a hydrogaph. 

In [ ]:
def rising_falling_main(multiplier, days_ahead, input_gauge_df, input_sat_passes):
    ''' Split the satellite passes into either the rising or falling category '''

    rising_list = list()
    falling_list = list()
    
    for i, flow in enumerate(input_gauge_df['Value'][:len(input_gauge_df['Value'])-days_ahead]):
        if flow < input_gauge_df['Value'][i + days_ahead] * multiplier:
            rising_list.append(input_gauge_df.index[i])
        else:
            falling_list.append(input_gauge_df.index[i])
    
    rising_passes = list(set(rising_list) & set(list(input_sat_passes.index)))
    falling_passes = list(set(falling_list) & set(list(input_sat_passes.index)))

    rising_passes_df = df_constructor(rising_passes, input_sat_passes)
    falling_passes_df = df_constructor(falling_passes, input_sat_passes)
    
    return rising_passes_df, falling_passes_df

def df_constructor(input_list, sat_data_to_join):
    ''' converts the list of either rising or falling passes into a dataframe '''
    
    df = pd.DataFrame(input_list, columns = ['date'])
    df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
    df = df.set_index('date')
    df = df.join(sat_data_to_join)
    
    return df

multiplier = 1 # how much more the flow rate has to rise by to be considered a significant rise
days_ahead = 21 # how many days in advance the algorithm checks for a rise or fall

ls_rising, ls_falling = rising_falling_main(multiplier, days_ahead, clean_gauge_data, ls_all_merged_data)
s_rising, s_falling = rising_falling_main(multiplier, days_ahead, clean_gauge_data, s_all_merged_data)

In [ ]:
def graph_rising_falling(input_gauge_data, input_rising_ls, input_falling_ls, input_rising_s, 
                         input_falling_s):
    ''' Function to generate a graph showing the satellite passes relative to where they occur
    on the hydrograph and their respective category (either rising or falling)'''    
    
    graph_gauge_df = add_flow_bands_to_gauge_data(input_gauge_data)
    
    fig_rising_falling = go.Figure()
    
    fig_rising_falling.add_trace(go.Scatter(name = 'Gauge data',
                                           x = graph_gauge_df.index,
                                           y = graph_gauge_df['Value'],
                                           mode = 'lines',
                                           line = dict(color = 'royalblue', width = 1),
                                           hovertemplate = 'flow: %{y:.2f}<extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>'
                                           )
                                )
    
    fig_rising_falling.add_trace(go.Scatter(name = 'Rising Landsat pass',
                                           x = input_rising_ls.index,
                                           y = input_rising_ls['Value'],
                                           mode = 'markers',
                                           marker_symbol = 'triangle-up',
                                           marker_size = 9,
                                           line = dict(color = 'green', width = 2),
                                           hovertemplate = 'Rising Landsat pass <extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>'
                                           )
                                ) 
    
    fig_rising_falling.add_trace(go.Scatter(name = 'Falling Landsat pass',
                                           x = input_falling_ls.index,
                                           y = input_falling_ls['Value'],
                                           mode = 'markers',
                                           marker_symbol = 'triangle-down',
                                           marker_size = 9,
                                           line = dict(color = 'crimson', width = 2),
                                           hovertemplate = 'Falling Landsat pass <extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>'
                                           )
                                ) 
    
    fig_rising_falling.add_trace(go.Scatter(name = 'Rising Sentinel pass',
                                           x = input_rising_s.index,
                                           y = input_rising_s['Value'],
                                           mode = 'markers',
                                           marker_symbol = 'star-triangle-up',
                                           marker_size = 9,
                                           line = dict(color = 'lightseagreen', width = 2),
                                           hovertemplate = 'Rising Sentinel pass <extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>'
                                           )
                                ) 
        
    fig_rising_falling.add_trace(go.Scatter(name = 'Falling Sentinel pass',
                                           x = input_falling_s.index,
                                           y = input_falling_s['Value'],
                                           mode = 'markers',
                                           marker_symbol = 'star-triangle-down',
                                           marker_size = 9,
                                           line = dict(color = 'purple', width = 2),
                                           hovertemplate = 'Falling Sentinel pass <extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>'
                                           )
                                ) 
    
    # Adding the lower flow bounds:
    fig_rising_falling.add_trace(go.Scatter(name = 'lower flow bound',
                                x = graph_gauge_df.index,
                                y = graph_gauge_df['y_lower'],
                                mode = 'lines',
                                line = dict(color = 'black', width = 2)
                                )
                     )
    
    # Adding the upper flow bounds:
    fig_rising_falling.add_trace(go.Scatter(name = 'upper flow bound',
                                x = graph_gauge_df.index,
                                y = graph_gauge_df['y_higher'],
                                mode = 'lines',
                                line = dict(color = 'black', width = 2)
                                )
                     )
    
    fig_rising_falling.update_layout(hovermode="closest",
                                 title= 'Sentinel and Landsat broken into rising/falling categories',
                                 xaxis_title='Date',
                                 yaxis_title='Flow at gauge (ML/Day)',
                                 font=dict(family='Calibri', size=16, color='black'),
                                 paper_bgcolor='white',
                                 plot_bgcolor='white',
                                 hoverlabel=dict(
                                     bgcolor="white",
                                     font_size=16,
                                     font_family="Rockwell")
                                    ) 
    
    return fig_rising_falling


print("{} rising landsat passes".format(len(ls_rising)))
print("{} falling landsat passes".format(len(ls_falling)))
print("{} rising sentinel passes".format(len(s_rising)))
print("{} falling sentinel passes".format(len(s_falling)))
graph_rising_falling = graph_rising_falling(clean_gauge_data, ls_rising, ls_falling, s_rising, s_falling)
graph_rising_falling

In [ ]:
def rising_falling_cleaner(input_df, sat_source, pass_or_fail):
    ''' Ingests rising or falling dataframe,
    drops the columns not required for the rest of the analysis,
    adds column for the satellite and whether or not it will be rejected/passed'''
    
    df = input_df.copy(deep = True)
    
    df = df.drop(['spatial_ref_x', 'spatial_ref_y'], axis = 1)
    df['sat_result'] = str(sat_source + ' ' + pass_or_fail)
    
    return df

ls_rising_clean = rising_falling_cleaner(ls_rising, 'landsat', 'accept')
ls_falling_clean = rising_falling_cleaner(ls_falling, 'landsat', 'reject')
s_rising_clean = rising_falling_cleaner(s_rising, 'sentinel', 'accept')
s_falling_clean = rising_falling_cleaner(s_falling, 'sentinel', 'reject')
    
master_df = pd.concat([ls_rising_clean, ls_falling_clean, s_rising_clean, s_falling_clean])
ls_only = pd.concat([ls_rising_clean, ls_falling_clean])
s_only = pd.concat([s_rising_clean, s_falling_clean])

# PART 2 - Outputting the maps


## Manual check of satellite passes you would like to analyse
The program has split the satellite passes into the rising and falling limb and allocated them to a 'pass' and 'fail' category respectively. Only the satellite passes in the 'pass' category will be analysed. View the graph below, and click on the satellite passes you would like the reclassify, and the program will reclassify them for you.

In [ ]:
master_df_reindex = master_df.reset_index()

In [ ]:
def set_colour(x):
    '''Function to allocate a colour to the 
    sat pass depending on how it is classified '''
    if (x == 'landsat accept'):    
        return 'green'
    elif (x == 'landsat reject'):
        return 'red'
    elif (x == 'sentinel accept'):
        return 'purple'
    else:
        return 'lightpink' 

graph_gauge_df = add_flow_bands_to_gauge_data(clean_gauge_data)

# Adding landsat trace:
trace_sats = go.Scatter(name = 'Satellite pass',
                      x = master_df_reindex['date'],
                      y = master_df_reindex['Value'],
                      mode = 'markers',
                      marker = dict(size=8,
                                    color=list(map(set_colour, master_df_reindex['sat_result']))),
                      showlegend = False
                     )  

# Adding the gauge data:
trace_gauge = go.Scatter(name = 'Gauge data',
                     x = graph_gauge_df.index,
                     y = graph_gauge_df['Value'],
                     mode = 'lines',
                     line = dict(color = 'royalblue', width = 1),
                     hovertemplate = 'flow: %{y:.2f}<extra></extra>' +
                                            '<br><b>Date</b>: %{x}<br>')

# Adding the lower flow bounds:
trace_lower = go.Scatter(name = 'lower flow bound',
                     x = graph_gauge_df.index,
                     y = graph_gauge_df['y_lower'],
                     mode = 'lines',
                     line = dict(color = 'black', width = 2)
                    )

# Adding the upper flow bounds:
trace_upper = go.Scatter(name = 'upper flow bound',
                     x = graph_gauge_df.index,
                     y = graph_gauge_df['y_higher'],
                     mode = 'lines',
                     line = dict(color = 'black', width = 2)
                    )
# Adding for legend
trace_pass_ls = go.Scatter(name = 'landsat accepted',
                     x=[None], y=[None],
                     mode = 'markers',
                     marker = dict(size=8,
                                   color= 'green'),
                     showlegend = True)

# Adding for legend
trace_fail_ls = go.Scatter(name = 'landsat rejected', 
                     x=[None], y=[None],
                     mode= ' markers',
                     marker = dict(size=8,
                                   color= 'red'),
                     showlegend = True)

trace_pass_s = go.Scatter(name = 'sentinel accepted',
                     x=[None], y=[None],
                     mode = 'markers',
                     marker = dict(size=8,
                                   color= 'purple'),
                     showlegend = True)

# Adding for legend
trace_fail_s = go.Scatter(name = 'sentinel rejected', 
                     x=[None], y=[None],
                     mode= ' markers',
                     marker = dict(size=8,
                                   color= 'lightpink'),
                     showlegend = True)

# Add the traces to the figure:
passing_failing_graph = go.FigureWidget(data=[trace_sats, trace_gauge, 
                                              trace_lower, trace_upper, 
                                              trace_pass_ls, trace_fail_ls,
                                              trace_pass_s, trace_fail_s])

passing_failing_graph.update_layout(hovermode="closest",
                          title= 'Proposed satellite passes to use in the analysis',
                          xaxis_title='Date',
                          yaxis_title='Flow at gauge (ML/Day)',
                          font=dict(family='Calibri', size=16, color='black'),
                          paper_bgcolor='white',
                          plot_bgcolor='white',
                          hoverlabel=dict(
                              bgcolor="white",
                              font_size=16,
                              font_family="Rockwell")) 

scatter = passing_failing_graph.data[0]

def update_point(trace, points, selector):
    ''' Reclassify the points based on user selection '''
    
    try:
        date = points.xs[0]
        row_name = master_df_reindex.loc[master_df_reindex['date'] == date]
        
        if date in ls_only.index and date in s_only.index:
            if 'accept' in row_name['sat_result'].iloc[0]:
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'landsat reject'
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'sentinel reject'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the landsat and sentinel pass on ', date, ' has been reclassified to be rejected')
            else:
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'landsat accept'
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'sentinel accept'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the landsat and sentinel pass on ', date, ' has been reclassified to be accepted')
        
        elif date in ls_only.index and date not in s_only.index:
            if row_name['sat_result'].iloc[0] == 'landsat accept':
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'landsat reject'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the landsat pass on ', date, ' has been reclassified to be rejected')
            else:
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'landsat accept'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the landsat pass on ', date, ' has been reclassified to be accepted')
                
        elif date in s_only.index and date not in ls_only.index:
            if row_name['sat_result'].iloc[0] == 'sentinel accept':
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'sentinel reject'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the sentinel pass on ', date, ' has been reclassified to be rejected')
            else:
                master_df_reindex.loc[master_df_reindex['date'] == date, 'sat_result'] = 'sentinel accept'
                with passing_failing_graph.batch_update():
                    scatter.marker.color = list(map(set_colour, master_df_reindex['sat_result']))
                print('the sentinel pass on ', date, ' has been reclassified to be accepted')
                
    except IndexError:
        print('You missed the satellite pass! Try click again')   

scatter.on_click(update_point)

passing_failing_graph

In [ ]:
#Get only acceptable passes:
passes_only = master_df_reindex[master_df_reindex['sat_result'].str.contains('accept')]    

In [ ]:
dates_of_interest = passes_only['date'].values
sat_source = passes_only['sat_result'].values

for i, day in enumerate(dates_of_interest):
    
    print(i)   
    date_string = str(passes_only['date'].iloc[i].date())
    print(date_string)
    
    if sat_source[i] == 'landsat accept':

        file_extension = '.nc'
        file_name = 'netcdf_outputs/' + date_string + 'landsat' + file_extension
        # Create a reusable query
        landsat_data_query = {
            'x': (lon_low, lon_high),
            'y': (lat_low, lat_high),
            'time': (date_string),
            'measurements': ['nbart_red', 'nbart_green', 'nbart_blue', 
                             'nbart_nir', 'nbart_swir_1', 'nbart_swir_2'],
            'output_crs': 'EPSG:3577',
            'resolution': (-30, 30),
            'group_by': 'solar_day'
        }

        # Load available data from all three Landsat satellites
        ls_ds = load_ard(dc=landsat_dc, 
                      products=['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3'],
                      mask_pixel_quality=True,
                      min_gooddata=0.60,
                      ls7_slc_off=False, ## comment this to keep the images with the SLC error
                      **landsat_data_query)

        # Shows an RGB thumbnail of each pass:
        ls_ds[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=0).to_array().plot.imshow(robust=True, figsize=(8, 8))

        # Making an NDWI layer and showing a thumbnail of the output
        ds_ndwi = (ls_ds.nbart_nir - ls_ds.nbart_swir_1) /(ls_ds.nbart_nir + ls_ds.nbart_swir_1)
        ls_ds["NDWI"] = ds_ndwi
        ls_ds['NDWI'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        # Making the fisher index layer and showing a thumbnail of the output
        ds_fisher = 1.7204+(171*ls_ds.nbart_green)+(3*ls_ds.nbart_red)-(70*ls_ds.nbart_nir)-(45*ls_ds.nbart_swir_1)-(71*ls_ds.nbart_swir_2)
        ls_ds['Fisher'] = ds_fisher  
        ls_ds['Fisher'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        ds_burnsie = (ls_ds.nbart_swir_2 + ls_ds.nbart_nir + ls_ds.nbart_red)
        ls_ds['Burnsie'] = ds_burnsie 
        ls_ds['Burnsie'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        # Write the layers to a NetCDF, ready for importing
        try:
            write_dataset_to_netcdf(ls_ds, file_name)
        except RuntimeError:
            print('***THIS FILE NAME ALREADY EXISTS, PLEASE SAVE YOUR NETCDF FILES TO ANOTHER DIRECTORY***')
            break
            
    ########################################                
    elif sat_source[i] == 'sentinel accept':
        
        file_extension = '.nc'
        file_name = 'netcdf_outputs/' + date_string + 'sentinel' + file_extension
        # Create a reusable query
        sentinel_data_query = {
            'x': (lon_low, lon_high),
            'y': (lat_low, lat_high),
            'time': (date_string),
            'measurements': ['nbart_red', 'nbart_green', 'nbart_blue', 
                             'nbart_nir_1', 'nbart_swir_2', 'nbart_swir_3'],
            'output_crs': 'EPSG:3577',
            'resolution': (-10, 10),
            'group_by': 'solar_day'
        }

        # Load available data from all three Landsat satellites
        s_ds = load_ard(dc=sentinel_dc,
                      products=['s2a_ard_granule', 's2b_ard_granule'],
                      min_gooddata=0.10,
                      mask_pixel_quality=True,         
                      dask_chunks={},
                      **sentinel_data_query)

        # Shows an RGB thumbnail of each pass:
        s_ds[['nbart_red', 'nbart_green', 'nbart_blue']].isel(time=0).to_array().plot.imshow(robust=True, figsize=(8, 8))

        # Making an NDWI layer and showing a thumbnail of the output
        ds_ndwi = (s_ds.nbart_nir_1 - s_ds.nbart_swir_2) /(s_ds.nbart_nir_1 + s_ds.nbart_swir_2)
        s_ds["NDWI"] = ds_ndwi
        s_ds['NDWI'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        # Making the fisher index layer and showing a thumbnail of the output
        ds_fisher = 1.7204+(171*s_ds.nbart_green)+(3*s_ds.nbart_red)-(70*s_ds.nbart_nir_1)-(45*s_ds.nbart_swir_2)-(71*s_ds.nbart_swir_3)
        s_ds['Fisher'] = ds_fisher  
        s_ds['Fisher'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        ds_burnsie = (s_ds.nbart_swir_2 + s_ds.nbart_nir_1 + s_ds.nbart_red)
        s_ds['Burnsie'] = ds_burnsie 
        s_ds['Burnsie'].isel(time=0).plot.imshow(robust=True, figsize=(8, 8))

        # Write the layers to a NetCDF, ready for importing
        try:
            write_dataset_to_netcdf(s_ds, file_name)
            
        except RuntimeError:
            print('***THIS FILE NAME ALREADY EXISTS, PLEASE SAVE YOUR NETCDF FILES TO ANOTHER DIRECTORY***')
            break

#### Optional outputs 

In [ ]:
# Outputting hydrograph 1: all satellite flyovers on the gauge data:
py.plot(fig_all,filename='hydrograph_outputs/all_passes_plot.html')

In [ ]:
# Outputting hydrograph 2: satellite passes divided into rising and falling cats with gauge data
py.plot(graph_rising_falling,filename='hydrograph_outputs/rising_falling_plot.html')

In [ ]:
# Outputting hydrograph 3: satellite passes divided into the 
# final pass/fail categories with gauge data:
py.plot(passing_failing_graph, filename='hydrograph_outputs/pass_fail_plot.html')

In [ ]:
#Exporting the dataframe to the csv
master_df_reindex.to_csv("csv_outputs/all_fly_overs.csv")